# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Shi  ->  J. Shi  |  ['J. Shi']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
S. Kraus  ->  S. Kraus  |  ['S. Kraus']
J. Mathew  ->  J. Mathew  |  ['J. Mathew']
Arxiv has 56 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.13177


extracting tarball to tmp_2410.13177... done.
Retrieving document from  https://arxiv.org/e-print/2410.13353


extracting tarball to tmp_2410.13353...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2410.13353/AAmain.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors.tex' from 'tmp_2410.13353/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 164 bibliographic references in tmp_2410.13353/AAmain.bbl.
Issues with the citations
syntax error in line 1138: '=' expected
Retrieving document from  https://arxiv.org/e-print/2410.13457


extracting tarball to tmp_2410.13457... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.13353-b31b1b.svg)](https://arxiv.org/abs/2410.13353) | **Dynamical resonances in PHANGS galaxies**  |
|| M. Ruiz-García, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-10-18*|
|*Comments*| **|
|**Abstract**|            Bars are remarkable stellar structures that can transport gas toward centers and drive the secular evolution of galaxies. In this context, it is important to locate dynamical resonances associated with bars. For this study, we used ${Spitzer}$ near-infrared images as a proxy for the stellar gravitational potential and the ALMA CO(J=2-1) gas distribution from the PHANGS survey to determine the position of the main dynamical resonances associated with the bars in the PHANGS sample of 74 nearby star-forming galaxies. We used the gravitational torque method to estimate the location of the bar corotation radius ($R_{\rm CR}$), where stars and gas rotate at the same angular velocity as the bar. Of the 46 barred galaxies in PHANGS, we have successfully determined the corotation (CR) for 38 of them. The mean ratio of the $R_{\rm CR}$ to the bar radius ($R_{\rm bar}$) is $\mathcal{R} = R_{\rm CR}/R_{\rm bar} = 1.12$, with a standard deviation of $0.39$. This is consistent with the average value expected from theory and suggests that bars are predominantly fast. We also compared our results with other bar CR measurements from the literature, which employ different methods, and find good agreement ($\rho = 0.64$). Finally, using rotation curves, we have estimated other relevant resonances such as the inner Lindblad resonance (ILR) and the outer Lindblad resonance (OLR), which are often associated with rings. This work provides a useful catalog of resonances for a large sample of nearby galaxies and emphasizes the clear connection between bar dynamics and morphology.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.13177-b31b1b.svg)](https://arxiv.org/abs/2410.13177) | **Chemical abundances of 20 barium stars from the OHP spectra**  |
|| G. Yang, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2024-10-18*|
|*Comments*| *16 pages, 12 figures, accepted for publication in MNRAS*|
|**Abstract**|            Based on the high resolution and high signal-to-noise spectra, we derived the chemical abundances of 20 elements for 20 barium (Ba-) stars. For the first time, the detailed abundances of four sample stars, namely HD 92482, HD 150430, HD 151101 and HD 177304 have been analyzed. Additionally, Ba element abundance has been measured using high resolution spectra for the first time in six of the other 16 sample stars. Based on the [s/Fe] ratios, the Ba-unknown star HD 115927 can be classified as a strong Ba-star, while the Ba-likely star HD 160538 can be categorized into a mild Ba-star. Consequently, our sample comprises three strong and 17 mild Ba-stars. The light odd-Z metal elements and Fe-peak elements exhibit near-solar abundances. The [{\alpha}/Fe] ratios demonstrate decreasing trends with increasing metallicity. Moreover, the abundances of n-capture elements show significant enhancements in different degrees. Using a threshold of the signed distances to the solar r-process abundance pattern ds = 0.6, we find that all of our sample stars are normal Ba-stars, indicating that the enhancements of s-process elements should be attributed to material transfer from their companions. We compare the observed n-capture patterns of sample stars with the FRUITY models, and estimate the mass of the Thermally-Pulsing Asymptotic Giant Branch stars that previously contaminated the Ba-stars. The models with low masses can successfully explain the observations. From a kinematic point of view, we note that most of our sample stars are linked with the thin disk, while HD 130255 may be associated with the thick disk.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.13457-b31b1b.svg)](https://arxiv.org/abs/2410.13457) | **Large Interferometer For Exoplanets (LIFE). XIV. Finding terrestrial protoplanets in the galactic neighborhood**  |
|| L. Cesario, et al. -- incl., <mark>S. Kraus</mark>, <mark>J. Mathew</mark> |
|*Appeared on*| *2024-10-18*|
|*Comments*| *18 pages, 19 figures; accepted for publication in A&A*|
|**Abstract**|            The increased brightness temperature of young rocky protoplanets during their magma ocean epoch makes them potentially amenable to atmospheric characterization to distances from the solar system far greater than thermally equilibrated terrestrial exoplanets, offering observational opportunities for unique insights into the origin of secondary atmospheres and the near surface conditions of prebiotic environments. The Large Interferometer For Exoplanets (LIFE) mission will employ a space-based mid-infrared nulling interferometer to directly measure the thermal emission of terrestrial exoplanets. Here, we seek to assess the capabilities of various instrumental design choices of the LIFE mission concept for the detection of cooling protoplanets with transient high-temperature magma ocean atmospheres, in young stellar associations in particular. Using the LIFE mission instrument simulator (LIFEsim) we assess how specific instrumental parameters and design choices, such as wavelength coverage, aperture diameter, and photon throughput, facilitate or disadvantage the detection of protoplanets. We focus on the observational sensitivities of distance to the observed planetary system, protoplanet brightness temperature using a blackbody assumption, and orbital distance of the potential protoplanets around both G- and M-dwarf stars. Our simulations suggest that LIFE will be able to detect (S/N $\geq$ 7) hot protoplanets in young stellar associations up to distances of $\approx$100 pc from the solar system for reasonable integration times (up to $\sim$hours). Detection of an Earth-sized protoplanet orbiting a solar-sized host star at 1 AU requires less than 30 minutes of integration time. M-dwarfs generally need shorter integration times. The contribution from wavelength regions $<$6 $\mu$m is important for decreasing the detection threshold and discriminating emission temperatures.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.13353/./Appendix-All_galaxies-with_CR/IC1954-APPENDIX.png', 'tmp_2410.13353/./NGC1097-APPENDIX.png', 'tmp_2410.13353/./FancyR_comparison-QF12.png']
copying  tmp_2410.13353/./Appendix-All_galaxies-with_CR/IC1954-APPENDIX.png to _build/html/
copying  tmp_2410.13353/./NGC1097-APPENDIX.png to _build/html/


copying  tmp_2410.13353/./FancyR_comparison-QF12.png to _build/html/
exported in  _build/html/2410.13353.md
    + _build/html/tmp_2410.13353/./Appendix-All_galaxies-with_CR/IC1954-APPENDIX.png
    + _build/html/tmp_2410.13353/./NGC1097-APPENDIX.png
    + _build/html/tmp_2410.13353/./FancyR_comparison-QF12.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sqdiamond}[1][fill=black]{\tikz[x=1.2ex,y=1.85ex,line width=.1ex,line join=round, yshift=-0.285ex] \draw[#1]  (0,.5) -- (.5,1) -- (1,.5) -- (.5,0) -- (0,.5) -- cycle;}$
$\newcommand{\sqdiamondDash}[1][fill=black]{$
$    \tikz[x=1.2ex,y=1.85ex,line width=.1ex,line join=round, yshift=-0.285ex]$
$        \draw[#1]$
$            (0,.5) -- (.5,1) -- (1,.5) -- (.5,0) -- (0,.5) -- cycle$
$            (0,1.1) --  (1,1.1);$
$}$
$\newcommand{\MyDiamond}[1][fill=black]{\mathop{\raisebox{-0.275ex}{\sqdiamond[#1]}}}$
$\newcommand{\re}{\ensuremath{r_\mathrm{e}}}$
$\newcommand{\SigStar}{\ensuremath{\Sigma_\mathrm{\star}}}$
$\newcommand{\SigMol}{\ensuremath{\Sigma_\mathrm{mol}}}$
$\newcommand{\SigSFR}{\ensuremath{\Sigma_\mathrm{SFR}}}$
$\newcommand{\tdep}{\ensuremath{\tau_\mathrm{dep}}}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IPAC}{\label{IPAC} Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}$
$\newcommand{\Carnegie}{\label{Carnegi} Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{ÇAPP}{\label{CCAPP} Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\CITEVA}{\label{CITEVA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CNRS}{\label{CNRS} CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\Heidelberg}{\label{Heidelberg} Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\COOL}{\label{COOL} Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\ICRAR}{\label{ICRAR} International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}$
$\newcommand{\IRAM}{\label{IRAM} Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\JHU}{\label{JHU} Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\Leiden}{\label{Leiden} Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\MPE}{\label{MPE} Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\Nagoya}{\label{Nagoya} Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}$
$\newcommand{\NRAO}{\label{NRAO} National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ObsParis}{\label{ObsParis} Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, 4 Ivy Ln., Princeton, NJ 08544 USA}$
$\newcommand{\UToledo}{\label{UToledo} University of Toledo, 2801 W. Bancroft St., Mail Stop 111, Toledo, OH, 43606}$
$\newcommand{\Toulouse}{\label{Toulouse} Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\UChile}{\label{UChile} Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}$
$\newcommand{\UConn}{\label{UConn} Department of Physics, University of Connecticut, Storrs, CT, 06269, USA}$
$\newcommand{\UCSD}{\label{UCSD} Center for Astrophysics and Space Sciences, Department of Physics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UCSDAA}{\label{UCSDAA} Department of Astronomy \& Astrophysics,  University of California, San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UMass}{\label{UMass} University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LAM}{\label{LAM} Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille), Marseille, France}$
$\newcommand{\UHawaii}{\label{UHawaii} Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\IPARC}{\label{IPARC} Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\McMaster}{\label{McMaster} Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\INAF}{\label{INAF} INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}$
$\newcommand{\Sydney}{\label{Sydney} Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}$
$\newcommand{\UA}{\label{UA} Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}$
$\newcommand{\CITA}{\label{CITA} Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George St, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\ASIAA}{\label{ASIAA} Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 10617, Taiwan}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\PSMA}{\label{PSMA} Penn State Mont Alto, 1 Campus Drive, Mont Alto, PA  17237, USA}$
$\newcommand{\ILL}{\label{ILL} Institut Laue-Langevin, 71 avenue des Martyrs, F-38042 Grenoble, France}$
$\newcommand{\TUM}{\label{TUM} Technical University of Munich, School of Engineering and Design, Department of Aerospace and Geodesy, Chair of Remote Sensing Technology, Arcisstr. 21, 80333 Munich, Germany}$
$\newcommand{\Surrey}{\label{Surrey} Department of Physics, University of Surrey, Guildford GU2 7XH, UK}$
$\newcommand{\Oxford}{\label{Oxford} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\AIP}{\label{AIP} Leibniz-Institut for Astrophysik Potsdam (AIP), An der Sternwarte 16, 14482 Potsdam, Germany}$
$\newcommand{\Insubria}{\label{Insubria}{Universit{à} dell’Insubria, via Valleggio 11, 22100 Como, Italy}}$
$\newcommand{\StAndrews}{\label{StAndrews} School of Physics and Astronomy, University of St Andrews, North Haugh, St Andrews, KY16 9SS}$
$\newcommand{\IAC}{\label{IAC}{Instituto de Astrofísica de Canarias, C/ Vía Láctea s/n, E-38205, La Laguna, Spain}}$
$\newcommand{\ULL}{\label{ULL}{Departamento de Astrofísica, Universidad de La Laguna, Av. del Astrofísico Francisco Sánchez s/n, E-38206, La Laguna, Spain}}$
$\newcommand{\AIfA}{\label{AIfA}{$
$Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}}$
$\newcommand{\Heidel}{\label{Heidel} Universität Heidelberg, Zentrum für Astronomie, Albert-Ueberle-Str. 2, 69120 Heidelberg, Germany}$
$\newcommand{\UMich}{\label{UMich}$
$Department of Astronomy, University of Michigan, Ann Arbor, MI 48109, USA}$
$\newcommand{\Msun}{\ifmmode{\mathrm M_\odot}\else{M_\odot}\fi}$
$\newcommand\action{#1}$
$\newcommand\change{#1}$</div>



<div id="title">

# Dynamical resonances in PHANGS galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.13353-b31b1b.svg)](https://arxiv.org/abs/2410.13353)<mark>Appeared on: 2024-10-18</mark> - 

</div>
<div id="authors">

M. Ruiz-García, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Bars are remarkable stellar structures that can transport gas toward centers and drive the secular evolution of galaxies. In this context, it is important to locate dynamical resonances associated with bars. For this study, we used _Spitzer_ near-infrared images as a proxy for the stellar gravitational potential and the ALMA CO(J=2-1) gas distribution from the PHANGS survey to determine the position of the main dynamical resonances associated with the bars in the PHANGS sample of 74 nearby star-forming galaxies. We used the gravitational torque method to estimate the location of the bar corotation radius ( $R_{\rm CR}$ ), where stars and gas rotate at the same angular velocity as the bar. Of the 46 barred galaxies in PHANGS, we have successfully determined the corotation (CR) for 38 of them. The mean ratio of the $R_{\rm CR}$ to the bar radius ( $R_{\rm bar}$ ) is $\mathcal{R} = R_{\rm CR}/R_{\rm bar} = 1.12$ , with a standard deviation of $0.39$ . This is consistent with the average value expected from theory and suggests that bars are predominantly fast. We also compared our results with other bar CR measurements from the literature, which employ different methods, and find good agreement ( $\rho = 0.64$ ). Finally, using rotation curves, we have estimated other relevant resonances such as the inner Lindblad resonance (ILR) and the outer Lindblad resonance (OLR), which are often associated with rings. This work provides a useful catalog of resonances for a large sample of nearby galaxies and emphasizes the clear connection between bar dynamics and morphology.

</div>

<div id="div_fig1">

<img src="tmp_2410.13353/./Appendix-All_galaxies-with_CR/IC1954-APPENDIX.png" alt="Fig17" width="100%"/>

**Figure 17. -** **IC 1954**(SB, $QF=3$) non-weighted deprojected torque map (left panel), torque profile (upper central panel), comparison with values from the literature (lower central panel), rotation curve (upper right panel), and angular rotation curve (lower right panel). The cyan-contoured ellipse indicates the bar extent. We show contours corresponding to $[5\sigma, 15\sigma, 45\sigma, ..., 0.9\sigma_{\rm max}]$ in the CO map, where $\sigma$ is the mean value of the gas map and $\sigma_{\rm max}$ its maximum value. CR (if determined) is represented as a vertical pink line, together with its uncertainties (pink shaded area). This is a statistical uncertainty due to bootstrapping for $i$, PA and center position (see Sect. $\re$f{sec:bootstrap}). Solid green line represents the bar length (from \citealt{Querejeta+21}), while shaded green region represents the region where we search for the CR. Brown dashed line represents the radius at which the coverage of CO starts to be non-uniform ($R_{\rm 100\%  CO}$ in Table $\re$f{table:complete_sample}), orange dashed line is the radius at which the coverage of CO is uniform about 50\%($R_{\rm 50\%  CO}$ in Table $\re$f{table:complete_sample}), and purple dashed line represents the end of CO coverage ($R_{\rm End  CO}$ in Table $\re$f{table:complete_sample}). Teal dashed line represents the nuclear bar registered by \cite{Querejeta+21}(if detected). The shaded gray region shown in this graphic represents the inner region inside which we cannot say anything on $\tau(R)$ due to the limited spatial resolution of our observations. In addition, on the lower central panel, each dot represents a different measure of the CR (of the main bar) from literature. Finally, on the upper right panel we show the rotation curve from \cite{Lang+20}, while on the lower right panel we represent in black solid line the angular rotation curve ($\Omega$) and the curves derived as $\Omega \pm \kappa/2$(light pink and blue curves), together with the derived pattern speed ($\Omega_p$) and its uncertainties, shown as a dotted pink line and a pink-filled region. In this panel we also represent the Lindblad Resonances if present, as two shades of blue (iILR and oILR) and orange (OLR) dotted lines. (*fig:Appendix-All_galaxies-IC1954*)

</div>
<div id="div_fig2">

<img src="tmp_2410.13353/./NGC1097-APPENDIX.png" alt="Fig11" width="100%"/>

**Figure 11. -** **NGC 1097 (SB)** non-weighted deprojected torque map (left panel), torque profile (upper central panel), comparison with values from the literature (lower central panel), rotation curve (upper right panel), and angular rotation curve (lower right panel). The cyan-contoured ellipse in the left panel indicates the bar extent. We show contours corresponding to $[5\sigma, 15\sigma, 45\sigma, \dots, 0.9\sigma_{\rm max}]$ in the CO map, where $\sigma$ is the mean value of the gas map $\sigma = 0.59 \rm K  km  s^{-1}$, and $\sigma_{\rm max} = 934.83  \rm K  km  s^{-1}$. In the central and right panels, CR is represented as a vertical pink line, together with its uncertainties (pink-shaded area). This is a statistical uncertainty due to bootstrapping for $i$, PA and center position (see Sect. $\re$f{sec:bootstrap}). The solid green line represents the bar length (from \citealt{Querejeta+21}), while the shaded green region represents the region where we search for the CR. Brown dashed line marks the radius at which the coverage of CO starts to be nonuniform ($R_{\rm 100 \% CO}$ in Table $\re$f{table:complete_sample}), orange dashed line is the radius at which the coverage of CO is uniform about $50\%$($R_{\rm 50 \% CO}$ in Table $\re$f{table:complete_sample}), and purple dashed line represents the end of CO coverage ($R_{\rm End  CO}$ in Table $\re$f{table:complete_sample}). The shaded gray region represents the inner region inside which we cannot say anything on $\tau(R)$ due to the limited spatial resolution of our observations. Finally, in the lower central panel, each dot represents a different measure of the CR (of the main bar) from the literature, and the $\MyDiamond$ symbol represents a measure of a nuclear bar CR. For both right panels, the solid black line represents the rotation curve (upper panel) and the angular rotation curve $\Omega$(lower panel). Solid light pink and light blue lines represent $\Omega + \kappa /2$ and $\Omega - \kappa /2$, respectively. The OLR and its uncertainties are represented in orange and the oILR (and its uncertainties) in dark blue. Purple vertical line represents the central ring detected by \cite{Querejeta+21}. (*fig:NGC1097-case_study*)

</div>
<div id="div_fig3">

<img src="tmp_2410.13353/./FancyR_comparison-QF12.png" alt="Fig3" width="100%"/>

**Figure 3. -** Slow, fast, and ultra-fast bars containing exclusively galaxies with $QF = 1$ and $QF = 2$. Galaxies with $QF = 2$ are represented with stars, while galaxies with $QF = 1$ are represented with dots. (*fig:QF1&2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.13353"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

222  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
